Make an MT5 indicator that highlights the corresponding candle red if bearish and blue if bullish on the very first candlestick formed at 0:00. Use MQL5, do not ever mix MT4-style functions in MT5, do not ever use the MT4-style helper. 

#property indicator_chart_window
#property strict

#property indicator_buffers 1
#property indicator_plots   1
#property indicator_type1   DRAW_COLOR_CANDLES
#property indicator_color1  Blue
#property indicator_width1  2
#property indicator_style1  STYLE_SOLID

double OpenBuffer[];
double HighBuffer[];
double LowBuffer[];
double CloseBuffer[];

datetime LastProcessedDay = 0;

//+------------------------------------------------------------------+
//| Custom indicator initialization function                         |
//+------------------------------------------------------------------+
int OnInit()
{
   // Set buffers
   SetIndexBuffer(0, CloseBuffer, INDICATOR_DATA);
   PlotIndexSetInteger(0, PLOT_DRAW_TYPE, DRAW_CANDLES);
   PlotIndexSetInteger(0, PLOT_LINE_COLOR, clrBlue);

   // Remove grid and enable Bid/Ask lines
   ChartSetInteger(0, CHART_SHOW_GRID, false);
   ChartSetInteger(0, CHART_SHOW_BID, true);
   ChartSetInteger(0, CHART_SHOW_ASK, true);

   return(INIT_SUCCEEDED);
}

//+------------------------------------------------------------------+
//| Custom indicator iteration function                                |
//+------------------------------------------------------------------+
int OnCalculate(const int rates_total,
                const int prev_calculated,
                const int begin,
                const double &price[])
{
   if(rates_total < 1)
      return(0);

   for(int i = 0; i < rates_total; i++)
   {
      datetime time_i = Time[i];
      MqlDateTime dt;
      TimeToStruct(time_i, dt);

      // Only process the first candle of each day (0:00)
      if(dt.hour == 0 && dt.min == 0)
      {
         // Only process new days
         if(LastProcessedDay != dt.day)
         {
            LastProcessedDay = dt.day;

            // Set candle colors based on bullish/bearish
            if(Close[i] > Open[i])
            {
               // Bullish
               PlotIndexSetInteger(0, PLOT_LINE_COLOR, clrBlue);
            }
            else if(Close[i] < Open[i])
            {
               // Bearish
               PlotIndexSetInteger(0, PLOT_LINE_COLOR, clrRed);
            }
            else
            {
               PlotIndexSetInteger(0, PLOT_LINE_COLOR, clrGray); // Neutral
            }
         }
      }
   }

   return(rates_total);
}
